# Bulls Analytics - Exploration Notebook

This notebook is for exploring Chicago Bulls data, finding insights, and creating visualizations.

## Workflow
1. **Explore** - Fetch data and see what's interesting
2. **Analyze** - Calculate trends, averages, comparisons
3. **Visualize** - Create charts and graphics

In [ ]:
# Setup - Import libraries and configure
import sys
from pathlib import Path

# Add parent directory to path so we can import bulls
sys.path.insert(0, str(Path().absolute().parent))

from bulls import data, analysis, viz
from bulls.config import CURRENT_SEASON
import pandas as pd
import matplotlib.pyplot as plt

# Configure matplotlib for better display in notebooks
plt.style.use('default')
%matplotlib inline

print("✅ Bulls Analytics loaded")
print(f"📅 Season: {CURRENT_SEASON}")
print(f"📊 Ready to explore!")

## Quick Exploration: Last Game

Let's dive into the most recent Bulls game and find something interesting!

In [ ]:
# Get the latest game and explore it
game = data.get_latest_game()
box = data.get_box_score(game['game_id'])

# === OVERVIEW ===
print("=" * 60)
print("🏀 LAST GAME OVERVIEW")
print("=" * 60)
print(f"\n📅 Date: {game['date']}")
print(f"🎯 Matchup: {game['matchup']}")
print(f"📍 Location: {'Home' if game['is_home'] else 'Away'}")
print(f"🏆 Result: {'✅ WIN' if game['result'] == 'W' else '❌ LOSS'}")

# Calculate opponent score
opponent_score = game['bulls_score'] - game['plus_minus']
print(f"📊 Final Score: Bulls {game['bulls_score']} - {opponent_score} {game['opponent']}")
print(f"📈 Point Differential: {game['plus_minus']:+.1f}")

# Team stats summary
if not box.empty:
    total_points = box['points'].sum()
    total_rebounds = box['reboundsTotal'].sum()
    total_assists = box['assists'].sum()
    total_fg_made = box['fieldGoalsMade'].sum()
    total_fg_attempted = box['fieldGoalsAttempted'].sum()
    total_fg3_made = box['threePointersMade'].sum()
    total_fg3_attempted = box['threePointersAttempted'].sum()
    
    fg_pct = (total_fg_made / total_fg_attempted * 100) if total_fg_attempted > 0 else 0
    fg3_pct = (total_fg3_made / total_fg3_attempted * 100) if total_fg3_attempted > 0 else 0
    
    print(f"\n📈 Team Stats:")
    print(f"   Total Points: {total_points}")
    print(f"   Total Rebounds: {total_rebounds}")
    print(f"   Total Assists: {total_assists}")
    print(f"   Field Goal %: {fg_pct:.1f}% ({total_fg_made}/{total_fg_attempted})")
    print(f"   3-Point %: {fg3_pct:.1f}% ({total_fg3_made}/{total_fg3_attempted})")

# Top performers
top = analysis.top_performers(box)
if top:
    print(f"\n⭐ Top 3 Performers:")
    for i, player in enumerate(top[:3], 1):
        print(f"   {i}. {player['name']}: {player['points']} PTS, {player['rebounds']} REB, {player['assists']} AST")

# === INTERESTING STAT ===
print("\n" + "=" * 60)
print("💡 INTERESTING STAT")
print("=" * 60)

if not box.empty and len(top) > 0:
    # Find the most efficient scorer (points per field goal attempt)
    efficiency_stats = []
    for player in top:
        if player['fg_attempted'] > 0:
            efficiency = player['points'] / player['fg_attempted']
            efficiency_stats.append({
                'name': player['name'],
                'points': player['points'],
                'fg_attempted': player['fg_attempted'],
                'efficiency': efficiency
            })
    
    if efficiency_stats:
        # Sort by efficiency
        efficiency_stats.sort(key=lambda x: x['efficiency'], reverse=True)
        most_efficient = efficiency_stats[0]
        
        print(f"\n🎯 Most Efficient Scorer: {most_efficient['name']}")
        print(f"   Points per Shot Attempt: {most_efficient['efficiency']:.2f}")
        print(f"   ({most_efficient['points']} points on {most_efficient['fg_attempted']} FGA)")
        
        # Compare to team average
        team_avg_efficiency = total_points / total_fg_attempted if total_fg_attempted > 0 else 0
        diff = most_efficient['efficiency'] - team_avg_efficiency
        print(f"   Team Average: {team_avg_efficiency:.2f} points per FGA")
        print(f"   Difference: {diff:+.2f} ({'+' if diff > 0 else ''}{(diff/team_avg_efficiency*100):.1f}% above team avg)")

print("\n" + "=" * 60)

## 1. Recent Games

Get the last N Bulls games and see what's happening.

In [ ]:
# Get last 10 Bulls games
games = data.get_games(last_n=10)

# Display key columns
display(games[['GAME_DATE', 'MATCHUP', 'WL', 'PTS', 'PLUS_MINUS']])

# Quick summary
wins = (games['WL'] == 'W').sum()
losses = (games['WL'] == 'L').sum()
print(f"\n📈 Last 10 games: {wins}W - {losses}L")
print(f"📊 Average points: {games['PTS'].mean():.1f}")
print(f"📊 Average +/-: {games['PLUS_MINUS'].mean():+.1f}")

## 2. Latest Game

Get the most recent game and box score.

In [ ]:
# Get latest game
game = data.get_latest_game()

print(f"🏀 {game['matchup']}")
print(f"📅 {game['date']}")
print(f"🏆 Result: {game['result']}")
print(f"📊 Score: Bulls {game['bulls_score']}, Opponent {game['bulls_score'] - game['plus_minus']}")
print(f"📍 {'Home' if game['is_home'] else 'Away'} game")

# Get box score
box = data.get_box_score(game['game_id'])

# Show top performers
top = analysis.top_performers(box)
print(f"\n⭐ Top Performers:")
for i, player in enumerate(top[:5], 1):
    print(f"{i}. {player['name']}: {player['points']} PTS, {player['rebounds']} REB, {player['assists']} AST")

# Display full box score
display(box[['name', 'points', 'reboundsTotal', 'assists', 'steals', 'blocks', 'fieldGoalsMade', 'fieldGoalsAttempted']].sort_values('points', ascending=False))

## 3. Player Analysis

Analyze a specific player's performance over recent games.

In [ ]:
# Change this to analyze different players
PLAYER_NAME = "Coby White"
LAST_N_GAMES = 15

# Get player's recent games
player_games = data.get_player_games(PLAYER_NAME, last_n=LAST_N_GAMES)

if not player_games.empty:
    # Display game log
    display(player_games[['date', 'matchup', 'result', 'points', 'rebounds', 'assists', 'fg_pct', 'fg3_pct']])
    
    # Calculate averages
    avgs = analysis.season_averages(player_games)
    print(f"\n📊 {PLAYER_NAME} - Averages over {avgs['games']} games:")
    print(f"   Points: {avgs['points']:.1f} PPG")
    print(f"   Rebounds: {avgs['rebounds']:.1f} RPG")
    print(f"   Assists: {avgs['assists']:.1f} APG")
    print(f"   FG%: {avgs['fg_pct']:.1f}%")
    print(f"   3PT%: {avgs['fg3_pct']:.1f}%")
    
    # Analyze trend
    trend = analysis.scoring_trend(player_games)
    print(f"\n📈 Scoring Trend: {trend['direction'].upper()}")
    print(f"   Recent avg: {trend['recent_avg']:.1f} PPG")
    print(f"   Overall avg: {trend['average']:.1f} PPG")
    print(f"   High: {trend['high']} | Low: {trend['low']}")
else:
    print(f"❌ No games found for {PLAYER_NAME}")

## 4. Visualizations

Create charts to visualize player performance and trends.

In [ ]:
# Bar chart of scoring
if not player_games.empty:
    fig = viz.bar_chart(
        player_games,
        x='date',
        y='points',
        title=f"{PLAYER_NAME} - Points Per Game (Last {LAST_N_GAMES})"
    )
    plt.show()
else:
    print("No data to visualize. Run the player analysis cell first.")

In [ ]:
# Line chart showing trend
if not player_games.empty:
    fig = viz.line_chart(
        player_games,
        x='date',
        y='points',
        title=f"{PLAYER_NAME} - Scoring Trend"
    )
    plt.show()
else:
    print("No data to visualize. Run the player analysis cell first.")

## 5. Create Visualizations

When you've found an interesting insight, create visualizations to share.

In [ ]:
# Example: Create additional visualizations
# Customize these based on your analysis

# You can create multiple chart types to explore different aspects
# of the data and find interesting insights

## Tips

- **Explore first**: Use the data fetching cells to see what's interesting
- **Ask questions**: "How has player X been performing?" "What's the trend?"
- **Visualize**: Create charts to see patterns
- **Find the story**: Look for unique angles (clutch performance, trends, comparisons)

## Next Steps

1. Try different players
2. Compare multiple players
3. Look for interesting patterns (home vs away, vs specific teams, etc.)
4. Create custom visualizations
5. Experiment with different chart types to find insights